In [2]:
import os
import keras
from keras.layers import Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from keras.models import Sequential
from scipy.io import wavfile as wav
import librosa
import librosa.display
import numpy as np
import pandas as pd
import random
import struct
import matplotlib.pyplot as plt
import IPython.display as ipd
import progressbar
import time

In [3]:
data = pd.read_csv('metadata/Screamsound.csv')
valid_data = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] > 0 ]
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + '/' + valid_data['slice_file_name'].astype('str')


In [16]:
# 读取wav文件函数
def path_class(filename):
    excerpt = data[data['slice_file_name'] == filename]
    path_name = os.path.join('audio', 'fold'+str(excerpt.fold.values[0]), filename)
    return path_name, excerpt['class'].values[0]


In [32]:
# 读取wav声音文件，并提取器mfcc特征，以及label标签，将其保存
bar = progressbar.ProgressBar(maxval=data.shape[0], widgets=[progressbar.Bar('$', '||', '||'), ' ', progressbar.Percentage()])
bar.start()
dataset = [[0 for x in range(1)] for x in range(data.shape[0])]
for i in range(data.shape[0]):
    
    fullpath, class_id = path_class(data.slice_file_name[i])
    try:
        X, sample_rate = librosa.load(fullpath, res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
        print(mfccs)
        print (type(mfccs))
    except Exception:
        print("Error encountered while parsing file: ", file)
        mfccs,class_id = None, None
    feature = mfccs
    label = class_id
    
    print(dataset)
    print (type(dataset))
    
    dataset[i,0],dataset[i,1] = feature,label
    
    print(dataset)
    print (type(dataset))
    
    bar.update(i+1)


[-2.76074005e+02  1.01702332e+02 -9.09617081e+01 -4.58845758e+00
 -1.91535026e-01 -1.27734118e+01 -1.72550297e+01  3.26614213e+00
 -9.61936474e+00  1.53714705e+01  8.93650055e+00  1.23601656e+01
  1.45476198e+00  2.06550193e+00 -6.15444422e+00  2.80930114e+00
  3.33138108e+00  1.68819785e+00 -1.29912674e+00  8.02487850e+00
  6.12269640e+00  1.19570926e-01  5.47339725e+00 -1.02392741e-01
 -1.36843956e+00  4.32706308e+00  5.81786871e-01  2.44431996e+00
  2.54816842e+00  6.51461482e-01  4.73611259e+00  1.25436807e+00
 -1.09259188e+00  6.34501219e-01  1.84880817e+00  1.79349935e+00
  2.03636026e+00  9.41113383e-02  2.66597056e+00  8.53925765e-01]
<class 'numpy.ndarray'>
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
<class 'list'>


TypeError: list indices must be integers or slices, not tuple

In [3]:
from tqdm import tnrange, tqdm_notebook

D=[]
        
for row in tqdm_notebook(valid_data.itertuples()): 
    print(row.path)
    print(row.classID)
    y1, sr1 = librosa.load("audio/" + row.path, duration=2.97)  
    ps = librosa.feature.melspectrogram(y=y1, sr=sr1)
    print(ps.shape)
    if ps.shape != (128, 128): 
            continue
    D.append( (ps, row.classID) )

C:\Users\USER\AppData\Local\Temp\ipykernel_8412\2348564150.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(valid_data.itertuples()):


0it [00:00, ?it/s]

fold2/100652-3-0-0.wav
1
(128, 128)
fold2/100652-3-0-1.wav
1
(128, 128)
fold2/100652-3-0-2.wav
1
(128, 128)
fold2/100652-3-0-3.wav
1
(128, 128)
fold2/101415-3-0-2.wav
1
(128, 128)
fold2/101415-3-0-3.wav
1
(128, 128)
fold2/101415-3-0-8.wav
1
(128, 128)
fold1/yisell_sound_2007_11_9_1_31_156634.wav
0
(128, 80)
fold1/yisell_sound_2007_12_18_15_25_315033.wav
0
(128, 13)
fold1/yisell_sound_2007_12_18_15_25_593398.wav
0
(128, 34)
fold1/yisell_sound_2007_12_18_15_26_248752.wav
0
(128, 69)
fold1/yisell_sound_2007_12_18_15_26_444257.wav
0
(128, 84)
fold1/yisell_sound_2007_12_18_15_27_063699.wav
0
(128, 62)
fold1/yisell_sound_2007_12_18_15_27_324809.wav
0
(128, 94)
fold1/yisell_sound_2007_12_18_15_27_543156.wav
0
(128, 128)
fold1/yisell_sound_2007_12_18_15_29_067921.wav
0
(128, 128)
fold1/yisell_sound_2007_12_18_15_29_33722.wav
0
(128, 128)
fold1/yisell_sound_2007_12_18_15_30_231364.wav
0
(128, 128)
fold1/yisell_sound_2014080511023849341_66366.wav
0
(128, 88)
fold1/yisell_sound_201408051232386395

In [17]:
import numpy as np
import hickle as hkl 
data = D

# Dump data to file
hkl.dump(data, 'new_data_file.hkl')

# 将声音文件的mfcc特征以及label保存，便于之后的导入
np.save("dataset",dataset,allow_pickle=True)


In [5]:
# Load data from file
D = hkl.load('new_data_file.hkl')

In [6]:
from keras.utils import np_utils
dataset = D
random.shuffle(dataset)

train = dataset[:8]
test = dataset[8:]
print('dataset:', len(dataset))
print('train:', len(train))
print('test:', len(test))

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])


y_train = np.array(keras.utils.np_utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.np_utils.to_categorical(y_test, 10))

dataset: 12
train: 8
test: 4


In [15]:
import numpy as np
import pandas as pd

# 导入数据
data = pd.DataFrame(np.load("dataset.npy",allow_pickle= True))
data.columns = ['feature', 'label']

# 对数据进行处理
from sklearn.preprocessing import LabelEncoder

X = np.array(data.feature.tolist())
y = np.array(data.label.tolist())

# 数据分割
from sklearn.model_selection import train_test_split
X,val_x,y,val_y = train_test_split(X,y)

# 对标签进行one-hot处理
lb = LabelEncoder()
from keras.utils import np_utils
y = np_utils.to_categorical(lb.fit_transform(y))
val_y = np_utils.to_categorical(lb.fit_transform(val_y))


NameError: name 'y' is not defined

In [11]:
"""odel = Sequential()
input_shape=(128, 128, 1)

model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(MaxPooling2D((4, 2), strides=(4, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (5, 5), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(10))
model.add(Activation('softmax'))
 
model.summary()"""

'model = Sequential()\ninput_shape=(128, 128, 1)\n\nmodel.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=input_shape))\nmodel.add(MaxPooling2D((4, 2), strides=(4, 2)))\nmodel.add(Activation(\'relu\'))\n\nmodel.add(Conv2D(48, (5, 5), padding="valid"))\nmodel.add(MaxPooling2D((4, 2), strides=(4, 2)))\nmodel.add(Activation(\'relu\'))\n\nmodel.add(Conv2D(48, (5, 5), padding="valid"))\nmodel.add(Activation(\'relu\'))\n\nmodel.add(Flatten())\nmodel.add(Dropout(rate=0.5))\n\nmodel.add(Dense(64))\nmodel.add(Activation(\'relu\'))\nmodel.add(Dropout(rate=0.5))\n\nmodel.add(Dense(10))\nmodel.add(Activation(\'softmax\'))\n \nmodel.summary()'

In [8]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=12,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
1/1 [==============================] - 1s 856ms/step - loss: 42.6125 - accuracy: 0.0000e+00 - val_loss: 4.0255 - val_accuracy: 0.5000
Epoch 2/12
1/1 [==============================] - 0s 119ms/step - loss: 13.8385 - accuracy: 0.0000e+00 - val_loss: 3.2431 - val_accuracy: 0.2500
Epoch 3/12
1/1 [==============================] - 0s 120ms/step - loss: 18.1635 - accuracy: 0.2500 - val_loss: 3.7665 - val_accuracy: 0.2500
Epoch 4/12
1/1 [==============================] - 0s 145ms/step - loss: 2.5496 - accuracy: 0.5000 - val_loss: 5.9241 - val_accuracy: 0.2500
Epoch 5/12
1/1 [==============================] - 0s 169ms/step - loss: 5.1248 - accuracy: 0.6250 - val_loss: 6.9809 - val_accuracy: 0.2500
Epoch 6/12
1/1 [==============================] - 0s 114ms/step - loss: 5.0861 - accuracy: 0.6250 - val_loss: 7.7343 - val_accuracy: 0.2500
Epoch 7/12
1/1 [==============================] - 0s 110ms/step - loss: 3.1174 - accuracy: 0.6250 - val_loss: 8.2700 - val_accuracy: 0.2500
Epoch 8/1

In [9]:
from keras.models import load_model
timestr = time.strftime('%Y%m%d-%H%M%S')
modelName = 'sound-classification-{}.h5'.format(timestr)
model.save('models/{}'.format(modelName))

print('Model exported and finished')
model.save('my_model.h5') 

Model exported and finished
